In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import resource
from datetime import datetime

## Run times analyis

In [2]:
DATA_FOLDER = "../saved_ll/"

In [16]:
def time_file(file, total_runs):
    subprocess.run(["clang-7", 
                        "-o", "res", 
                        file, 
                        "-mllvm", "-enable-misched=false",
                        "-lm",
                        ])
    pid = os.getpid()
    os.sched_setaffinity(pid, {0})
    times = []
    for i in range(total_runs):
        usage_start = resource.getrusage(resource.RUSAGE_CHILDREN)
        subprocess.run(["./res"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        usage_end = resource.getrusage(resource.RUSAGE_CHILDREN)
        times.append(usage_end.ru_utime - usage_start.ru_utime)
        
    min_time = min(times)
    avg_time = np.mean(times)
    std_time = np.std(times)
    os.sched_setaffinity(pid, {0, 1, 2, 3})
    subprocess.run(["rm", "res"])
    
    return {"min": min_time, "avg": avg_time, "std": std_time}

In [27]:
run_times = {file: {} for file in os.listdir(DATA_FOLDER)}

In [28]:
total_runs = 10
for file in run_times:
    res = time_file(f"{DATA_FOLDER}{file}", total_runs)
    now = datetime.now()
    date = now.strftime("%d/%m %H:%M")
    run_times[file][(date, total_runs)] = res

In [29]:
run_times

{'0.4608.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.00031349999999999987,
   'std': 0.0009404999999999996}},
 '0.5112.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.001328600000000002,
   'std': 0.0016273370394604826}},
 '0.4471.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.0010268000000000054,
   'std': 0.0015713471799701118}},
 '0.4508.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.001583800000000002,
   'std': 0.0015865216544377833}},
 '0.455.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.0013145000000000073,
   'std': 0.0016119047273334829}},
 '0.5143.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.0005789999999999962,
   'std': 0.0012117445275304408}},
 '0.4446.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.0006435999999999997,
   'std': 0.0012888549336523489}},
 '0.4457.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.0009452000000000016,
   'std': 0.001446856786278447}},
 '0.4989.ll': {('24/03 11:46', 10): {'min': 0.0,
   'avg': 0.000985